In [1]:
import pandas as pd 
import os

current_directory = os.getcwd()

df = pd.read_excel(current_directory+"\Sample - Superstore.xlsx")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country/Region,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2020-152156,2020-11-08,2020-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2020-138688,2020-06-12,2020-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2019-108966,2019-10-11,2019-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [2]:
df1 = df.copy()
df1.columns


Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country/Region', 'City',
       'State', 'Postal Code', 'Region', 'Product ID', 'Category',
       'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount',
       'Profit'],
      dtype='object')

In [3]:
df1['Year'] = df1['Order Date'].dt.year
df1 = df1[['Year','Sales','Profit','Sub-Category']]
df_sort = df1.sort_values(by = ['Sub-Category','Year'])
df_sort = df_sort.groupby(['Sub-Category','Year']).sum().reset_index()

# Meassure YOY Sales and Change in profit features

df_sort['profit_last_year'] = df_sort['Profit'].shift(1) 
df_sort['Change in profit (2020 to 2021)'] = (df_sort.Profit - df_sort.profit_last_year).round(2)

df_sort['Sales_last_year'] = df_sort.Sales.shift(1)
df_sort['YOY Sales %'] = ((df_sort.Sales - df_sort.Sales_last_year) *100/ df_sort.Sales_last_year).round(2)

# Filter to only 2021 year
df_sort = df_sort[df_sort.Year == 2021].reset_index(drop = True)

# Crearte rank features
df_sort['Rank Profit'] =  df_sort['Profit'].rank(ascending=False)
df_sort['Rank Sales'] =  df_sort['Sales'].rank(ascending=False)

#rename the features
df_sort.rename(columns = {'Sub-Category' : 'Item','Sales':'2021 Sales','Profit':'2021 Profit'},inplace = True)

df_final = df_sort[['Item','2021 Sales','2021 Profit','YOY Sales %','Change in profit (2020 to 2021)','Rank Profit','Rank Sales']]


#Save to new file csv
df_final.to_csv(current_directory+"\Superstore.csv",index= False)

In [4]:
df_final

,Item,2021 Sales,2021 Profit,YOY Sales %,Change in profit (2020 to 2021),Rank Profit,Rank Sales
0,Accessories,59946.2320,15672.3570,43.08,6008.07,2.0,7.0
1,Appliances,42926.9320,7865.2683,64.78,2563.93,5.0,9.0
2,Art,8863.0680,2221.9631,48.69,808.00,10.0,14.0
3,Binders,72788.0450,7669.7418,46.50,-2545.89,6.0,3.0
4,Bookcases,30024.2797,-583.6261,14.27,-796.10,14.0,10.0
5,Chairs,95554.3530,7643.5493,13.87,1880.40,7.0,2.0
6,Copiers,62899.3880,25031.7902,26.81,7289.00,1.0,5.0
7,Envelopes,3378.5740,1441.7590,-28.57,-624.99,12.0,16.0
8,Fasteners,857.5940,304.9489,-10.68,10.88,13.0,17.0
9,Furnishings,28915.0940,4099.1628,3.73,163.89,9.0,11.0
